In [1]:

var IsotonicRegression = require('eclairjs/mllib/regression/IsotonicRegression');
var IsotonicRegressionModel = require('eclairjs/mllib/regression/IsotonicRegressionModel');
var FloatRDD = require('eclairjs/FloatRDD');
var Tuple2 = require('eclairjs/Tuple2');
var Tuple3 = require('eclairjs/Tuple3');


In [2]:

    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');
    var sparkConf = new SparkConf().setAppName("Isotonic Regression Example");
    var sc = new SparkContext(sparkConf);

var filename =  "../../data/mllib/sample_isotonic_regression_data.txt";
    var data = sc.textFile(filename);


 Create label, feature, weight tuples from input data with weight set to default value 1.0.


In [3]:
    var parsedData = data.map(function (line, Tuple3) {
        var parts = line.split(",");
        return new Tuple3(parseFloat(parts[0]), parseFloat(parts[1]), 1.0);
    }, [Tuple3]);


 Split data into training (60%) and test (40%) sets.


In [4]:
    var splits = parsedData.randomSplit([0.6, 0.4], 11);
    var training = splits[0];
    var test = splits[1];


 Create isotonic regression model from training data.
 Isotonic parameter defaults to true so it is only shown for demonstration


In [5]:
    var model = new IsotonicRegression().setIsotonic(true).run(training);


 Create tuples of predicted and real labels.


In [6]:
    var predictionAndLabel = test.mapToPair(function (point, model, Tuple2) {
        var predictedLabel = model.predict(point._2());
        return new Tuple2(predictedLabel, point._1());

    }, [model, Tuple2]);


 Calculate mean squared error between predicted and real labels.


In [7]:
    var meanSquaredError = new FloatRDD(predictionAndLabel.map(function (pl) {
        return Math.pow(pl._1() - pl._2(), 2);
    })).mean();

    var result = {};
    result.meanSquaredError = meanSquaredError;
    result.model = model;
    print("Mean Squared Error = " + result.meanSquaredError);


 Save and load model


In [8]:
    result.model.save(sc, "target/tmp/myIsotonicRegressionModel");
    var sameModel = IsotonicRegressionModel.load(sc, "target/tmp/myIsotonicRegressionModel");


In [9]:
    sc.stop();
